# COGS 108 - Data Checkpoint

# Names

- Nathaniel Wong
- Ethan Tan
- Judy Liu
- Clara Pozuelos
- Aidan Twul

<a id='research_question'></a>
# Research Question

Was the performance of technology related sectors as represented by the performance of the QQQ as well as physical entertainment companies as represented by the Las Vegas Sands (LVS) corporation in the stock market directly influenced by the rise and fall of COVID-19 infection rates in the 2020 year?

# Dataset(s)

- Dataset Name: QQQ Historical Data
- Link to the dataset: https://finance.yahoo.com/quote/QQQ/history?period1=1568592000&period2=1644537600&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true
- Number of observations: 604
QQQ: This data is information about the open and close of the Invesco QQQ Trust Series 1 Fund which tracks majoor technology stocks. We will be using the information from this stock to track the general sentiment of technology companies in relation to corona virus infections.

- Dataset Name: LVS Historical Data
- Link to the dataset: https://finance.yahoo.com/quote/LVS/history?period1=1568592000&period2=1644537600&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true
- Number of observations: 609
LVS: This data is information about the open and close of Las Vegas Sands company, a major casino entertainment conglomerate. We will be using the information from this data to track the general sentiment about physical entertainment companies in relation to corona virus infections.

- Dataset Name: WHO Covid-19 Global Data
- Link to the dataset: https://data.humdata.org/dataset/coronavirus-covid-19-cases-and-deaths
- Number of observations: 182727
WHO: This data is information about the Coronavirus COVID-19 daily new and cumulative cases and deaths globally. We will be using the information from this data to track the number of cases of Covid-19 in the United States during the 2020 year.

Because we are tracking the relationship between covid-19 infections and the performance of stocks in technology and physical enteretainment sectors, we will ideally combining the data of covid infection rates along with the QQQ and LVS performance data over time to see if there is a general correlation between rising infection rates with rising technology stock performance, and a general correlation between rising infection rates with decreasing phsyical entertainment stock perforamcne.

# Setup

In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest
import matplotlib.pyplot as plt

In [2]:
QQQ_df = pd.read_csv('./datasets/QQQ.csv')
LVS_df = pd.read_csv('./datasets/LVS.csv')
WHO_df = pd.read_csv('./datasets/WHO-COVID-19-global-data.csv')

In [3]:
QQQ_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-09-16,191.300003,192.080002,191.110001,191.679993,188.889801,20319000
1,2019-09-17,191.699997,192.600006,191.289993,192.600006,189.796417,20953400
2,2019-09-18,192.199997,192.600006,190.110001,192.520004,189.717560,29781300
3,2019-09-19,192.860001,194.039993,192.470001,192.839996,190.032898,26039400
4,2019-09-20,193.110001,193.309998,190.160004,190.800003,188.022614,29086100


In [4]:
LVS_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-09-16,59.049999,59.759998,58.919998,59.639999,57.074806,3266200
1,2019-09-17,58.070000,59.160000,57.230000,59.110001,57.307491,5347800
2,2019-09-18,58.900002,59.000000,57.700001,58.380001,56.599754,2629000
3,2019-09-19,58.599998,58.939999,57.349998,57.580002,55.824150,3807300
4,2019-09-20,57.650002,57.869999,56.270000,56.330002,54.612267,4804300


In [5]:
WHO_df.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0


# Data Cleaning

In regards to cleaning up the QQQ and LVS stock data, there was minimal steps we had to undertake 
because the data we collected from Yahoo Finance provided everything we needed in a concise manner 
without any extraneous data that had to be cleaned up.

In [6]:
QQQ_df.isna().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [7]:
LVS_df.isna().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In regards to cleaning up the WHO Covid-19 data, the data was already cleaned because they were provided by a service that directly affiliates with World Health Organization. Due to our need, we have filtered the dataset so we only look at cases from the US and cases that happened in 2020.

In [8]:
# Check for country by unique name
WHO_df['Country'].value_counts

<bound method IndexOpsMixin.value_counts of 0         Afghanistan
1         Afghanistan
2         Afghanistan
3         Afghanistan
4         Afghanistan
             ...     
182722       Zimbabwe
182723       Zimbabwe
182724       Zimbabwe
182725       Zimbabwe
182726       Zimbabwe
Name: Country, Length: 182727, dtype: object>

In [9]:
# Create a new Dataframe with just cases reagrding the US
us_covid_df = WHO_df[WHO_df['Country'] == 'United States of America']

# Checking to see if any information is missing in the Dataframe
us_covid_df.isna().sum()

Date_reported        0
Country_code         0
Country              0
WHO_region           0
New_cases            0
Cumulative_cases     0
New_deaths           0
Cumulative_deaths    0
dtype: int64

In [10]:
# Change the 'Date_reported' column to actual datetime type data 
# The below is wrong, keep getting this error: A value is trying to be set on a 
# copy of a slice from a DataFrame. Try using .loc[row_indexer,col_indexer] = value instead
us_covid_df['Date_reported'] = pd.to_datetime(us_covid_df['Date_reported'])

# Rechecking that the column types are correct
us_covid_df.dtypes

/var/folders/80/3fbmvq71367fgg085czl0scr0000gn/T/ipykernel_42894/4186771931.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_covid_df['Date_reported'] = pd.to_datetime(us_covid_df['Date_reported'])


Date_reported        datetime64[ns]
Country_code                 object
Country                      object
WHO_region                   object
New_cases                     int64
Cumulative_cases              int64
New_deaths                    int64
Cumulative_deaths             int64
dtype: object

In [11]:
# Checking the column names of the dataframe
us_covid_df.columns

Index(['Date_reported', 'Country_code', 'Country', 'WHO_region', 'New_cases',
       'Cumulative_cases', 'New_deaths', 'Cumulative_deaths'],
      dtype='object')

In [12]:
# Selecting only the necessary columns needed to do our analysis
us_covid_df = us_covid_df[['Date_reported', 'Country', 'New_cases', 'Cumulative_cases', 
                          'New_deaths', 'Cumulative_deaths']]

# Reconfiguring dataframe to include dates only in the year 2020
us_covid_df.loc[us_covid_df['Date_reported'].dt.year == 2020]

,Date_reported,Country,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
174246,2020-01-03,United States of America,0,0,0,0
174247,2020-01-04,United States of America,0,0,0,0
174248,2020-01-05,United States of America,0,0,0,0
174249,2020-01-06,United States of America,0,0,0,0
174250,2020-01-07,United States of America,0,0,0,0
...,...,...,...,...,...,...
174605,2020-12-27,United States of America,158525,18894000,2226,343083
174606,2020-12-28,United States of America,149358,19043358,1650,344733
174607,2020-12-29,United States of America,152831,19196189,1890,346623
174608,2020-12-30,United States of America,178998,19375187,2226,348849
